## Using only 2 out of the 12 REST tools

In [ ]:
# ONE‑CELL: REST‑wrapped tools ➜ Azure OpenAI function‑calling agent
# -----------------------------------------------------------------
# %pip install --quiet "openai>=1.25.0" requests python-dotenv

import os, json, uuid, requests
from dotenv import load_dotenv
from openai import AzureOpenAI

# 0. config --------------------------------------------------------
load_dotenv()

# Define the url of the Lex backend
LEX = "http://localhost:8000"
MODEL = "gpt-4o"

# Define the Azure OpenAI client
oai = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_KEY"),
    api_version = "2025-04-01-preview",
)

# 1. REST wrappers -------------------------------------------------
# functions that wrap the REST API endpoints exposed by Lex that can be called by the LLM
def rest_search_legislation_section(query: str, size: int = 3):
    r = requests.post(f"{LEX}/legislation/section/search",
                      json={"query": query, "size": size}, timeout=30)
    r.raise_for_status()
    return r.json()

def rest_search_caselaw_section(query: str, size: int = 3):
    r = requests.post(f"{LEX}/caselaw/section/search",
                      json={"query": query, "size": size}, timeout=30)
    r.raise_for_status()
    return r.json()

# dictionary that maps the function names to the actual functions
LOCAL_ROUTER = {
    "search_legislation_section": rest_search_legislation_section,
    "search_caselaw_section":     rest_search_caselaw_section,
}

# 2. tool specs ----------------------------------------------------
# define the tools that the LLM can use, ie the functions that can be called by the LLM
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "search_legislation_section",
            "description": "Search UK legislation sections.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"},
                    "size":  {"type": "integer", "default": 3},
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_caselaw_section",
            "description": "Search court‑judgment sections that cite legislation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"},
                    "size":  {"type": "integer", "default": 3},
                },
                "required": ["query"],
            },
        },
    },
]

# 3. agent loop ----------------------------------------------------
def chat(question: str) -> str:
    """
    Ask a question to the LLM. It might make three tool calls at once, all of which are executed,
    and based on the results, it can decide to either make more tool calls or not.
    """
    system_prompt = """
    You are a UK public-law assistant. Approach each question systematically:
    - First, use search tools to explore the topic broadly across legislation, case law, and explanatory notes
    - Then, use lookup tools for any specific acts, cases, or provisions you discover
    - Make follow-up searches based on what you learn
    - Continue gathering information until you can provide a comprehensive answer backed by multiple sources
    - Use tools liberally - it's better to over-research than under-research
    """

    msgs = [
        {"role": "system",
         "content": system_prompt},
        {"role": "user", "content": question},
    ]
    # if the assistant stops calling tools, return the answer
    while True:
        resp = oai.chat.completions.create(
            model = MODEL,
            messages = msgs,
            # define the tools/functions that the LLM 'knows' it could use
            tools = TOOLS,
            # the LLM will decide which tool to use
            tool_choice = "auto",
        ).choices[0].message
        # if the LLM does not need to call a tool, return the answer
        if resp.tool_calls is None:
            return resp.content

        # Record the assistant message that requested tools
        msgs.append(resp)

        # Execute each requested tool and append its result
        for call in resp.tool_calls:
            # get the function name
            fn_name = call.function.name
            # get the arguments
            args = json.loads(call.function.arguments)
            # call the function with the arguments
            py_resp = LOCAL_ROUTER[fn_name](**args)
            # append the tool response to the messages
            msgs.append({
                "role": "tool",
                "tool_call_id": call.id,
                "name": fn_name,
                "content": json.dumps(py_resp),
            })
        # Once a batch of tool calls is done, show results to the LLM
        # and ask it to decide whether to call more tools or not

# 4. demo ----------------------------------------------------------
print(chat(
    # "Summarise the duty in section 5 of the Environment Act 2021 and cite any cases interpreting it."
    # "What duties does the Climate Change Act 2008 impose on the Secretary of State?"
    "what do we know about Mrs Staveley’s estate? You must quote relevant sections from the documents."

))


The following information regarding Mrs Staveley's estate has been drawn from both legislative texts and the judgment summaries:

---

### Legislative Summaries:
1. **No specific mention in legislative excerpts provided.**
   - The legislative texts retrieved did not explicitly address "Mrs Staveley's estate."

---

### Court Judgment Summaries:
The case record ([2020] UKSC 35) and relevant sections reveal:

1. **Divorce and Pension Transfer**:
   - Mrs. Staveley divorced in 2000. The settlement involved transferring her share from the company pension scheme established during her marriage (with Morayford Ltd) into a "section 32 buyout policy."
   - This policy had a clause that any unutilized funds in the pension upon her death would return to the company, potentially benefitting her ex-husband, which she found unacceptable.

   *“...her share of the company pension scheme be transferred to her...However, given the level of her salary with the company, the pension was over-funded, and

## Using all 12 REST tools

In [1]:
# ONE‑CELL: 12 REST tools, with tool use trace and look‑ups marked “requires exact ID”
# ----------------------------------------------------------------

import os, json, uuid, requests
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
LEX  = "http://localhost:8000"
MODEL = "gpt-4o"

oai = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_KEY"),
    api_version = "2025-04-01-preview",
)

# -------- endpoint table -------------------------------------------------
ENDPOINTS = {
    # --- searches (safe defaults) ---
    "/legislation/section/search":        ("search_legislation_section","Search UK legislation sections.", ["query"]),
    "/legislation/search":                ("search_legislation_act",   "Search UK legislation acts.", ["query"]),
    "/caselaw/section/search":            ("search_caselaw_section",   "Search caselaw sections.", ["query"]),
    "/caselaw/search":                    ("search_caselaw",           "Search caselaw judgments.", ["query"]),
    "/caselaw/reference/search":          ("search_caselaw_reference", "Find cases that cite a given case or legislation.", ["reference_id","reference_type"]),
    "/explanatory_note/section/search":   ("search_explanatory_note_section","Search explanatory note sections.",["query"]),
    "/amendment/search":                  ("search_amendment","Search amendments by legislation.", ["legislation_id"]),
    "/amendment/section/search":          ("search_amendment_section","Search amendments by provision.", ["provision_id"]),

    # --- look‑ups (need precise IDs) ---
    "/legislation/section/lookup":        ("lookup_legislation_sections","**Lookup** all sections of an Act (requires exact legislation_id).", ["legislation_id"]),
    "/legislation/lookup":                ("lookup_legislation","**Lookup** an Act by type + year + number (requires all three).", ["legislation_type","year","number"]),
    "/legislation/text":                  ("lookup_legislation_full_text","**Lookup** full text of an Act (requires exact legislation_id).", ["legislation_id"]),
    "/explanatory_note/legislation/lookup": ("lookup_explanatory_note","**Lookup** explanatory notes for an Act (requires exact legislation_id).",["legislation_id"]),
}

# -------- dynamic wrappers & tool specs ---------------------------------
LOCAL_ROUTER, TOOLS = {}, []

def make_wrapper(path, required):
    def _call(**body):
        missing = [k for k in required if k not in body]
        if missing:
            return {"error": f"missing {missing}"}
        try:
            r = requests.post(f"{LEX}{path}", json=body, timeout=60)
            r.raise_for_status()
            return r.json()
        except requests.HTTPError as e:
            return {"http_error": r.status_code, "detail": r.text[:300]}
    return _call

for path, (name, desc, req) in ENDPOINTS.items():
    LOCAL_ROUTER[name] = make_wrapper(path, req)

    props = {k: {"type": "string"} for k in req}
    if "size" not in req:
        props["size"] = {"type": "integer", "default": 10}

    TOOLS.append({
        "type": "function",
        "function": {
            "name": name,
            "description": desc,
            "parameters": {"type":"object",
                           "properties": props,
                           "required": req},
        },
    })

print(f"Registered {len(TOOLS)} tools.")

# ── agent loop with TRACE ──────────────────────────────────────────
def chat_trace(prompt: str, max_turns: int = 10) -> str:

    system_prompt = """
    You are a UK public‑law assistant. Approach each question systematically:
    - First, use search tools to explore the topic broadly across legislation, case law, and explanatory notes
    - Then, use lookup tools for any specific acts, cases, or provisions you discover
    - Make follow-up searches based on what you learn
    - Continue gathering information until you can provide a comprehensive answer backed by multiple sources
    - Use tools liberally - it's better to over-research than under-research.
    - Only call a look‑up tool when the user supplies an exact ID.
    """

    msgs = [
        {"role":"system",
         "content":system_prompt},
        {"role":"user","content":prompt},
    ]

    turn = 0
    while turn < max_turns:
        turn += 1
        resp = oai.chat.completions.create(
            model       = MODEL,
            messages    = msgs,
            tools       = TOOLS,
            tool_choice = "auto",
        ).choices[0].message

        # assistant chose to answer directly
        if resp.tool_calls is None:
            print(f"#F assistant's final response:\n{resp.content}...\n")
            return resp.content

        # log the assistant's tool requests
        msgs.append(resp)
        for call in resp.tool_calls:
            fn  = call.function.name
            arg = json.loads(call.function.arguments)
            print(f"#A assistant‑calls: {fn} {arg}")
            print("---"*10)

            tool_resp = LOCAL_ROUTER[fn](**arg)
            print(f"#T tool‑returns: {json.dumps(tool_resp)}...")
            print("---"*10)

            msgs.append({
                "role": "tool",
                "tool_call_id": call.id,
                "name": fn,
                "content": json.dumps(tool_resp),
            })

    # ran out of turns
    return "Reached max_turns without final answer."

# -------- demo -----------------------------------------------------------
print(chat_trace(
    "what do we know about Mrs Staveley’s estate?"
    # "Summarise the duty in section 5 of the Environment Act 2021 and cite any cases interpreting it."
    # "What duties does the Climate Change Act 2008 impose on the Secretary of State?"
))


Registered 12 tools.
#A assistant‑calls: search_caselaw_section {'query': "Mrs Staveley's estate", 'size': 10}
------------------------------
#T tool‑returns: [{"created_at": "2025-07-29T12:37:07.079131", "text": "21.Mrs Staveley was divorced in 2000. The divorce was acrimonious, leaving her feeling bitter towards her ex-husband. Whilst together, they had set up a company called Morayford Ltd. She was a director of the company, and employed by it, and she had a large pension fund with its occupational pension scheme. On divorce, her involvement with the company ceased. Putting it in the non-technical terms used by the FTT, \u201c[t]he ancillary relief order ordered that her share of the company pension scheme be transferred to her\u201d. It was put into what was called a \u201csection 32 buyout policy\u201d (the terminology reflecting that it was a policy to which section 32 of the Finance Act 1981 applied). However, given the level of her salary with the company, the pension was over-